## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.utils import resample
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.samplers import TPESampler
import joblib
import os
import shutil


### Varaibles important for pCR prediction

- Age
- Sex_M0_F1
- BMI
- cT
- cN
- CC_Length_cm
- Volume_cc
- EQD2_ab_10
- EQD2_ab_4.9
- Tech1
- Gap
- HPE_SqCC1_Adeno2 (exclude patient wit adenocarcinoma)
- Pre_Op_CTorPET_T_status
- Pre_Op_CTorPET_N_status
- Pre_Op_CTorPET_O_Status
- RTtoSurg_days
- Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4

## Prepraing Data

In [ ]:
import pandas as pd

# List of specified variables
selected_columns = {
    'no_NA': [
        'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 
        'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap', 'HPE_SqCC1_Adeno2',
        'RTtoSurg_days', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4', 'T_CR_Y0_N1', 'N_CR_Y0_N1', 
        'Overall_CR_Y0_N1'
    ],
    'no_PET': [
        'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'Volume_cc',
        'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap', 'HPE_SqCC1_Adeno2',
        'RTtoSurg_days', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4', 'T_CR_Y0_N1', 'N_CR_Y0_N1', 
        'Overall_CR_Y0_N1'
    ],
    'only_PET_overall': [
        'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'Volume_cc',
        'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap', 'HPE_SqCC1_Adeno2','Pre_Op_OverallCR_Y0_N1',
        'RTtoSurg_days', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4', 'T_CR_Y0_N1', 'N_CR_Y0_N1', 
        'Overall_CR_Y0_N1'
    ],
    'only_PET_overall_no_volume': [
        'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm','EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 
        'Gap', 'HPE_SqCC1_Adeno2','Pre_Op_OverallCR_Y0_N1', 'RTtoSurg_days', 
        'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4', 'T_CR_Y0_N1', 'N_CR_Y0_N1', 'Overall_CR_Y0_N1'
    ],
    'all': [
        'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'Volume_cc',
        'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap', 'HPE_SqCC1_Adeno2','Pre_Op_OverallCR_Y0_N1',
        'Pre_Op_CTPET_T_CR_Y0_N1', 'Pre_Op_CTPET_N_CR_Y0_N1',
        'RTtoSurg_days', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4', 'T_CR_Y0_N1', 'N_CR_Y0_N1', 
        'Overall_CR_Y0_N1'
    ]
}

# Columns to be used as labels
label_columns = ['T_CR_Y0_N1', 'N_CR_Y0_N1', 'Overall_CR_Y0_N1']



In [ ]:
# Load the data
df = pd.read_csv("data/RGCI_5_12_22.csv")
df_internal_val = pd.read_csv("data/RGCI_Int_Validate_25_1_23.csv")



In [ ]:
# Function to process datasets
def process_datasets(df, df_internal_val, selected_columns):
    # Select the specified columns
    df_selected = df[selected_columns].copy()
    df_internal_val_selected = df_internal_val[selected_columns].copy()
    
    # Remove rows with adenocarcinoma (HPE_SqCC1_Adeno2 != 2)
    df_selected = df_selected[df_selected['HPE_SqCC1_Adeno2'] != 2]
    df_internal_val_selected = df_internal_val_selected[df_internal_val_selected['HPE_SqCC1_Adeno2'] != 2]
    
    # Remove rows with any NA values
    df_selected = df_selected.dropna()
    df_internal_val_selected = df_internal_val_selected.dropna()

    # Check and align columns and data types
    columns_match = df_selected.dtypes.equals(df_internal_val_selected.dtypes)
    if not columns_match:
        print("There are differences in the columns or their properties between the datasets.")
        # Print the differences
        diff_df = pd.concat([df_selected.dtypes, df_internal_val_selected.dtypes], axis=1, keys=['Training', 'Validation'])
        diff_df = diff_df[diff_df['Training'] != diff_df['Validation']]
        print(diff_df)
        
        # Convert validation dataset columns to match training dataset data types
        for column in diff_df.index:
            if df_selected[column].dtype == 'int64' and df_internal_val_selected[column].dtype == 'float64':
                # Ensure the column can be safely converted to integers
                if df_internal_val_selected[column].dropna().apply(float.is_integer).all():
                    df_internal_val_selected[column] = df_internal_val_selected[column].astype('int64')
                else:
                    print(f"Column {column} contains non-integer floats and cannot be converted to int64.")
            else:
                df_internal_val_selected[column] = df_internal_val_selected[column].astype(df_selected[column].dtype)
        # Check again after conversion
        columns_match = df_selected.dtypes.equals(df_internal_val_selected.dtypes)
        if not columns_match:
            print("There are still differences in the columns or their properties between the datasets.")
            diff_df = pd.concat([df_selected.dtypes, df_internal_val_selected.dtypes], axis=1, keys=['Training', 'Validation'])
            diff_df = diff_df[diff_df['Training'] != diff_df['Validation']]
            print(diff_df)
        else:
            print("There are no differences in the columns or their properties between the datasets.")
            diff_df = pd.concat([df_selected.dtypes, df_internal_val_selected.dtypes], axis=1, keys=['Training', 'Validation'])
            diff_df = diff_df[diff_df['Training'] == diff_df['Validation']]
            print(diff_df)
    return df_selected, df_internal_val_selected

In [ ]:
# Process each dataset
df_selected_no_NA, df_internal_val_selected_no_NA = process_datasets(df, df_internal_val, selected_columns['no_NA'])
df_selected_no_PET, df_internal_val_selected_no_PET = process_datasets(df, df_internal_val, selected_columns['no_PET'])
df_selected_only_PET_overall_no_volume, df_internal_val_selected_only_PET_overall_no_volume = process_datasets(df, df_internal_val, selected_columns['only_PET_overall_no_volume'])
df_selected_only_PET_overall, df_internal_val_selected_only_PET_overall = process_datasets(df, df_internal_val, selected_columns['only_PET_overall'])
df_selected_all, df_internal_val_selected_all = process_datasets(df, df_internal_val, selected_columns['all'])



In [ ]:
# Separate features (X) and labels (y) for the validation set
X_val = df_internal_val_selected_no_NA.drop(columns=label_columns)
y_val = df_internal_val_selected_no_NA[label_columns]

# Display the shapes of the resulting datasets
print(f"Validation set: X_val shape = {X_val.shape}, y_val shape = {y_val.shape}")



In [ ]:
# Separate features (X) and labels (y) for each training set
training_sets = {
    'df_selected_no_NA': df_selected_no_NA,
    'df_selected_no_PET': df_selected_no_PET,
    'df_selected_only_PET_overall_no_volume': df_selected_only_PET_overall_no_volume,
    'df_selected_only_PET_overall': df_selected_only_PET_overall,
    'df_selected_all': df_selected_all
}

for name, df_train in training_sets.items():
    X_train = df_train.drop(columns=label_columns)
    y_train = df_train[label_columns]
    print(f"{name}: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")

## Preprocessing Pipeline

In [ ]:
# Define the feature categories
categorical_features = ['Sex_M0_F1','cT', 'cN','Tech1','HPE_SqCC1_Adeno2','Pre_Op_OverallCR_Y0_N1',
        'Pre_Op_CTPET_T_CR_Y0_N1', 'Pre_Op_CTPET_N_CR_Y0_N1','Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4']
continuous_features = ['Age','BMI','CC_Length_cm','Volume_cc','EQD2_ab_10','EQD2_ab_4.9','Gap','RTtoSurg_days']

In [ ]:
# Create pipelines for numerical and categorical features
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, continuous_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

## Creating features and labels for complete case analysis

In [ ]:
# Function to process datasets and separate features and labels
def process_and_separate(df, selected_columns, label_columns):
    # Select the specified columns
    df_selected = df[selected_columns].copy()
    
    # Remove rows with adenocarcinoma (HPE_SqCC1_Adeno2 != 2)
    df_selected = df_selected[df_selected['HPE_SqCC1_Adeno2'] != 2]
    
    # Remove rows with any NA values
    df_selected = df_selected.dropna()
    
    # Separate features (X) and labels (y)
    X = df_selected.drop(columns=label_columns)
    y1 = df_selected['T_CR_Y0_N1']
    y2 = df_selected['N_CR_Y0_N1']
    y3 = df_selected['Overall_CR_Y0_N1']
    
    return X, y1, y2, y3

In [ ]:
# Load the data
df = pd.read_csv("data/RGCI_5_12_22.csv")
df_internal_val = pd.read_csv("data/RGCI_Int_Validate_25_1_23.csv")

selected_columns = [
    'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'Volume_cc',
    'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap', 'HPE_SqCC1_Adeno2',
    'Pre_Op_OverallCR_Y0_N1', 'Pre_Op_CTPET_T_CR_Y0_N1',
    'Pre_Op_CTPET_N_CR_Y0_N1', 'RTtoSurg_days',
    'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4', 'T_CR_Y0_N1', 'N_CR_Y0_N1',
    'Overall_CR_Y0_N1'
]
# Columns to be used as labels
label_columns = ['T_CR_Y0_N1', 'N_CR_Y0_N1', 'Overall_CR_Y0_N1']



# Process the dataset and separate features and labels
X, y1, y2, y3 = process_and_separate(df, selected_columns, label_columns)
X_int_val, y1_int_val, y2_int_val, y3_int_val = process_and_separate(df_internal_val,selected_columns,label_columns)



In [ ]:
# Display the shapes of the resulting datasets
print(f"Features (X) shape = {X.shape}")
print(f"Label y1 shape = {y1.shape}")
print(f"Label y2 shape = {y2.shape}")
print(f"Label y3 shape = {y3.shape}")

print(f"Features (X_int_val) shape = {X_int_val.shape}")
print(f"Label y1_int_val shape = {y1_int_val.shape}")
print(f"Label y2_int_val shape = {y2_int_val.shape}")
print(f"Label y3_int_val shape = {y3_int_val.shape}")

# Model Building and HPO

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
roc_auc_score, roc_curve, auc, confusion_matrix, classification_report)
import optuna
from optuna.samplers import TPESampler

# Load the data
df = pd.read_csv("data/RGCI_5_12_22.csv")
df_internal_val = pd.read_csv("data/RGCI_Int_Validate_25_1_23.csv")

# Define the features and labels, excluding 'HPE_SqCC1_Adeno2'
selected_columns = [
    'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'Volume_cc',
    'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap',
    'Pre_Op_OverallCR_Y0_N1', 'Pre_Op_CTPET_T_CR_Y0_N1',
    'Pre_Op_CTPET_N_CR_Y0_N1', 'RTtoSurg_days',
    'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4'
]

label_columns = ['T_CR_Y0_N1', 'N_CR_Y0_N1', 'Overall_CR_Y0_N1']

# Define the categorical features
categorical_features = [
    'Sex_M0_F1', 'cT', 'cN', 'Tech1', 'Pre_Op_OverallCR_Y0_N1', 'Pre_Op_CTPET_T_CR_Y0_N1',
    'Pre_Op_CTPET_N_CR_Y0_N1', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4'
]



In [2]:
# Define the numerical features
numerical_features = list(set(selected_columns) - set(categorical_features))
# Filter patients with HPE_SqCC1_Adeno2 == 1 and drop this column
df_filtered = df[df['HPE_SqCC1_Adeno2'] == 1].drop(columns=['HPE_SqCC1_Adeno2'])

# Separate features (X) and labels (y)
X = df_filtered[selected_columns]
y1 = df_filtered['T_CR_Y0_N1']
y2 = df_filtered['N_CR_Y0_N1']
y3 = df_filtered['Overall_CR_Y0_N1']

In [3]:
# Define the numerical pipeline
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define the categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# model_pipeline = Pipeline([
#         ('preprocessor', preprocessor),
        # ('variance_threshold', VarianceThreshold()),  # Remove constant features
        # ('feature_selection', SelectKBest(score_func=f_classif, k=10)),
    #     ('classifier', RandomForestClassifier())
    # ])


# Hyperparameter Optimization for RandomForest

In [4]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y3, test_size=0.2, random_state=42)  # Using y3 as an label

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 10, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    # Define the feature selection and model pipeline
    
    rf_clf = make_pipeline(preprocessor, RandomForestClassifier(**params, random_state=42))
    cv_roc_auc = cross_val_score(rf_clf, X_train, y_train, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc

In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]
# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"RF_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_rf"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

In [ ]:
# loaded_study = optuna.create_study(study_name="RF_one_day_assessment_final04", storage=storage_name, load_if_exists=True)
# best_params = loaded_study.best_params

# Hyperparameter Optimization for XGBoost

In [ ]:
import xgboost as xgb
# HYPERPARAMETER OPTIMIZATION
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'lambda': trial.suggest_float('lambda', 0.0, 5.0),
        'alpha': trial.suggest_float('alpha', 0.0, 5.0)
    }
    xgb_clf = make_pipeline(preprocessor, xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss'))
    cv_roc_auc = cross_val_score(xgb_clf, X_train, y_train, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc

In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]

# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"XGB_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_xgb"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective_xgb, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

In [ ]:
storage_name = "sqlite:///db.sqlite3_xgb"
loaded_study = optuna.create_study(study_name="XGB_one_day_assessment_final01", storage=storage_name, load_if_exists=True)
best_params = loaded_study.best_params

# Hyperparameter Optimization for CatBoost

In [5]:
from catboost import CatBoostClassifier
def objective_cb(trial):
    params = {
    'iterations': trial.suggest_int('iterations', 50, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
    'depth': trial.suggest_int('depth', 1, 16),
    'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10),
    'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
    'border_count': trial.suggest_int('border_count', 32, 255),
    'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
    'rsm': trial.suggest_float('rsm', 0.5, 1.0),
    'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
    'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
    'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
    'od_wait': trial.suggest_int('od_wait', 10, 50),
    'verbose': 0  # Suppress output
}
    catboost_clf = make_pipeline(preprocessor, CatBoostClassifier(**params))
    cv_roc_auc = cross_val_score(catboost_clf, X_train, y_train, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc


In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]

# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"CatBoost_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_cb"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective_cb, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

[I 2024-08-09 06:44:46,711] A new study created in RDB with name: CatBoost_one_day_assessment_final_0


# Hyperparameter optimization for LightGBM

In [ ]:
def objective_lgbm(trial):
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
    'num_leaves': trial.suggest_int('num_leaves', 2, 256),
    'max_depth': trial.suggest_int('max_depth', -1, 50),  # -1 means no limit
    'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
    'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10),
    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
    'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
    'max_bin': trial.suggest_int('max_bin', 10, 500),
    'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
    'verbose': -1  # Suppress output
}

    lgbm_clf = make_pipeline(preprocessor, lgb.LGBMClassifier(**params))
    cv_roc_auc = cross_val_score(lgbm_clf, X_t, y_t, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc

In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]

# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"LGBM_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_lgbm"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective_lgbm, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

# Hyperparameter Optimization for SVM

In [ ]:
# # Nested cross-validation for hyperparameter optimization
def objective_svm(trial):
    params = {
        'C': trial.suggest_float('C', 0.1, 10.0),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
        'degree': trial.suggest_int('degree', 2, 5) if trial.params['kernel'] == 'poly' else 3,
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        'coef0': trial.suggest_float('coef0', 0.0, 1.0) if trial.params['kernel'] == 'poly' or trial.params['kernel'] == 'sigmoid' else 0.0
    }
    svc_clf = make_pipeline(preprocessor, SVC(**params, probability=True))
    cv_roc_auc = cross_val_score(svc_clf, X_t, y_t, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc

In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]

# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"SVM_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_svm"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective_svm, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

# Hyperparameter Optimization for Logistic Regression

In [ ]:
def objective_lr(trial):
    params = {
        'C': trial.suggest_float('C', 0.01, 10.0),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'sag', 'saga']),
        'max_iter': trial.suggest_int('max_iter', 100, 1000)
    }
    lr_clf = make_pipeline(preprocessor, LogisticRegression(**params))
    cv_roc_auc = cross_val_score(lr_clf, X_t, y_t, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc


In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]

# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"LR_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_lr"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective_lr, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

# Hyperparameter optimization for KNN

In [ ]:
# Nested cross-validation for hyperparameter optimization
def objective_knn(trial):
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 1, 50),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'algorithm': trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
        'leaf_size': trial.suggest_int('leaf_size', 10, 100),
        'p': trial.suggest_int('p', 1, 2)
    }
    knn_clf = make_pipeline(preprocessor, KNeighborsClassifier(**params))
    cv_roc_auc = cross_val_score(knn_clf, X_t, y_t, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc

In [ ]:
# List of random seeds
random_seeds  = [42, 50, 58, 66, 72]

# Loop over the random seeds
for i, seed in enumerate(random_seeds):
    study_name = f"KNN_one_day_assessment_final_{i}"
    storage_name = "sqlite:///db.sqlite3_knn"
    sampler = TPESampler(seed=seed)

    # Check if study exists, if not create a new one
    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    except KeyError:
        study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name=study_name)

    # Optimize the study
    study.optimize(objective_knn, n_trials=1000)

    # Print the best parameters and value for each run
    print(f"Study {i}: Best value: {study.best_value} (params: {study.best_params})")

# Cluster Analysis

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Define the features and labels
# selected_columns = [
#     'Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'Volume_cc',
#     'EQD2_ab_10', 'EQD2_ab_4.9', 'Tech1', 'Gap',
#     'Pre_Op_OverallCR_Y0_N1', 'Pre_Op_CTPET_T_CR_Y0_N1',
#     'Pre_Op_CTPET_N_CR_Y0_N1', 'RTtoSurg_days',
#     'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4'
# ]

selected_columns = ['Age', 'Sex_M0_F1', 'BMI', 'cT', 'cN', 'CC_Length_cm', 'EQD2_ab_10',
       'EQD2_ab_4.9', 'Tech1', 'Gap', 'Pre_Op_OverallCR_Y0_N1', 'RTtoSurg_days',
       'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4']

label_columns = ['T_CR_Y0_N1', 'N_CR_Y0_N1', 'Overall_CR_Y0_N1']

# Define the categorical features
# categorical_features = [
#     'Sex_M0_F1', 'cT', 'cN', 'Tech1', 'Pre_Op_OverallCR_Y0_N1', 'Pre_Op_CTPET_T_CR_Y0_N1',
#     'Pre_Op_CTPET_N_CR_Y0_N1', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4'
# ]

categorical_features = [
    'Sex_M0_F1', 'cT', 'cN', 'Tech1', 'Pre_Op_OverallCR_Y0_N1', 'Chemo_New_Cis_1_PC_2_Cis5F_3_Others_4'
]

# Load the data
df = pd.read_csv("data/RGCI_5_12_22.csv")
df = df.drop(columns=['Unnamed: 0'])
df_internal_val = pd.read_csv("data/RGCI_Int_Validate_25_1_23.csv")

# Filter patients with HPE_SqCC1_Adeno2 == 1 and drop this column
# df_filtered = df[df['HPE_SqCC1_Adeno2'] == 1].drop(columns=['HPE_SqCC1_Adeno2'])

# Separate features (X) and labels (y)
X = df_selected_only_PET_overall_no_volume[selected_columns]
y3 = df_selected_only_PET_overall_no_volume['Overall_CR_Y0_N1']

# Define the numerical features
numerical_features = list(set(selected_columns) - set(categorical_features))

In [ ]:
# df_selected_only_PET_overall_no_volume.drop(columns=['HPE_SqCC1_Adeno2'], inplace=True)

In [ ]:
# Define the numerical pipeline
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define the categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

In [ ]:
# Perform clustering
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X_preprocessed)

# Add cluster labels to the original dataframe
df_selected_only_PET_overall_no_volume['Cluster'] = clusters

# Analyze clusters with respect to y3
cluster_analysis = df_selected_only_PET_overall_no_volume.groupby('Cluster')[label_columns + ['Overall_CR_Y0_N1']].mean()
print(cluster_analysis)

# Chi-square test to check for significant differences in y3 across clusters
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_selected_only_PET_overall_no_volume['Cluster'], df_selected_only_PET_overall_no_volume['Overall_CR_Y0_N1'])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square test p-value: {p}")

In [ ]:
# Reduce dimensionality for visualization using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_preprocessed)

# Plot the clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters, palette='viridis')
plt.title('Clusters visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster')
plt.show()

# Visualize the relationship between clusters and y3
plt.figure(figsize=(10, 6))
sns.countplot(x='Cluster', hue='Overall_CR_Y0_N1', data=df_selected_only_PET_overall_no_volume)
plt.title('Cluster vs Pathological Complete Response (y3)')
plt.xlabel('Cluster')
plt.ylabel('Count')
plt.legend(title='Pathological Complete Response (y3)')
plt.show()

# Hierarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Perform hierarchical clustering
hierarchical = AgglomerativeClustering(n_clusters=5)
clusters_hierarchical = hierarchical.fit_predict(X_preprocessed)

# Add cluster labels to the original dataframe
df_selected_only_PET_overall_no_volume['Cluster_Hierarchical'] = clusters_hierarchical

# Analyze clusters with respect to y3
cluster_analysis_hierarchical = df_selected_only_PET_overall_no_volume.groupby('Cluster_Hierarchical')[label_columns + ['Overall_CR_Y0_N1']].mean()
print(cluster_analysis_hierarchical)

# Chi-square test to check for significant differences in y3 across clusters
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_selected_only_PET_overall_no_volume['Cluster_Hierarchical'], df_selected_only_PET_overall_no_volume['Overall_CR_Y0_N1'])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square test p-value: {p}")


In [ ]:
# Reduce dimensionality for visualization using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_preprocessed)

# Plot the clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters_hierarchical, palette='viridis')
plt.title('Clusters visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster Hierarchical')
plt.show()

# Visualize the relationship between clusters and y3
plt.figure(figsize=(10, 6))
sns.countplot(x='Cluster_Hierarchical', hue='Overall_CR_Y0_N1', data=df_selected_only_PET_overall_no_volume)
plt.title('Cluster vs Pathological Complete Response (y3)')
plt.xlabel('Cluster_Hierarchical')
plt.ylabel('Count')
plt.legend(title='Pathological Complete Response (y3)')
plt.show()

# Gaussian Mixed Model

In [ ]:
from sklearn.mixture import GaussianMixture

# Perform GMM clustering
gmm = GaussianMixture(n_components=3, random_state=42)
clusters_gmm = gmm.fit_predict(X_preprocessed)

# Add cluster labels to the original dataframe
df_selected_only_PET_overall_no_volume['Cluster_GMM'] = clusters_gmm

# Analyze clusters with respect to y3
cluster_analysis_gmm = df_selected_only_PET_overall_no_volume.groupby('Cluster_GMM')[label_columns + ['Overall_CR_Y0_N1']].mean()
print(cluster_analysis_gmm)

# Chi-square test to check for significant differences in y3 across clusters
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_selected_only_PET_overall_no_volume['Cluster_GMM'], df_selected_only_PET_overall_no_volume['Overall_CR_Y0_N1'])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square test p-value: {p}")

In [ ]:
# Reduce dimensionality for visualization using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_preprocessed)

# Plot the clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters_gmm, palette='viridis')
plt.title('Clusters visualization')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster GMM')
plt.show()

# Visualize the relationship between clusters and y3
plt.figure(figsize=(10, 6))
sns.countplot(x='Cluster_GMM', hue='Overall_CR_Y0_N1', data=df_selected_only_PET_overall_no_volume)
plt.title('Cluster vs Pathological Complete Response (y3)')
plt.xlabel('Cluster GMM')
plt.ylabel('Count')
plt.legend(title='Pathological Complete Response (y3)')
plt.show()